In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn.metrics
import re
import sklearn
import nltk
#nltk.download('omw-1.4')
#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import sys 
!{sys.executable} -m pip install tensorflow-addons
import contractions
from bs4 import BeautifulSoup
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score 
from sklearn.model_selection import train_test_split
import torch
#!pip install scikit-learn-intelex
from ast import literal_eval
from sklearnex import patch_sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
#!pip install pyxlsb
from sklearn import metrics


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

In [2]:
cleantf_df= pd.read_excel("ar-2010-2014-xlsb.xlsb")

tf_df= cleantf_df

cleanfs_df= pd.read_excel("ar-2015-2016-xlsb.xlsb")

fs_df= cleanfs_df

cleanse_df= pd.read_excel("ar-2017-2018.xlsx", sheet_name='Anonymised register')

se_df= cleanse_df




In [3]:
def old_ted_cleaning(fs_df):
    fs_df['total eggs mixed']= fs_df['total eggs mixed'].replace(['> 50'], 50)
    fs_df= fs_df.dropna(subset=['total eggs mixed'])
    fs_df['total eggs mixed'] = fs_df['total eggs mixed'].astype(int)
    fs_df['total eggs mixed'] = np.select(
        [fs_df['total eggs mixed'] == 0,
         fs_df['total eggs mixed'] .between(1, 5),
         fs_df['total eggs mixed'].between(6, 10),
         fs_df['total eggs mixed'].between(11, 15),
         fs_df['total eggs mixed'].between(16, 20),
         fs_df['total eggs mixed'].between(21, 25),
         fs_df['total eggs mixed'].between(26, 30),
         fs_df['total eggs mixed'].between(31, 35),
         fs_df['total eggs mixed'].between(35, 40),
         fs_df['total eggs mixed'] >= 40],
        ['0', '1', '2', '3', '4', '5','6','7','8','9'],
        np.nan
    )
    fs_df['fresh eggs collected']= fs_df['fresh eggs collected'].replace(['> 50'], 50)
    fs_df['fresh eggs collected'] = fs_df['fresh eggs collected'].fillna(0)
    fs_df['fresh eggs collected'] = fs_df['fresh eggs collected'].astype(int)
    fs_df['fresh eggs collected'] = np.select(
        [fs_df['fresh eggs collected'] == 0,
         fs_df['fresh eggs collected'] .between(1, 5),
         fs_df['fresh eggs collected'].between(6, 10),
         fs_df['fresh eggs collected'].between(11, 15),
         fs_df['fresh eggs collected'].between(16, 20),
         fs_df['fresh eggs collected'].between(21, 25),
         fs_df['fresh eggs collected'].between(26, 30),
         fs_df['fresh eggs collected'].between(31, 35),
         fs_df['fresh eggs collected'].between(35, 40),
         fs_df['fresh eggs collected'] >= 40],
        ['0', '1', '2', '3', '4', '5','6','7','8','9'],
        np.nan)
    return fs_df

In [4]:
def new_ted_cleaning(se_df):
    se_df['total eggs mixed']= se_df['total eggs mixed'].fillna(0)
    se_df['total eggs mixed']= se_df['total eggs mixed'].replace(['1-5'], 1)
    se_df['total eggs mixed']= se_df['total eggs mixed'].replace(['6-10'], 2)
    se_df['total eggs mixed']= se_df['total eggs mixed'].replace(['11-15'], 3)
    se_df['total eggs mixed']= se_df['total eggs mixed'].replace(['16-20'], 4)
    se_df['total eggs mixed']= se_df['total eggs mixed'].replace(['21-25'], 5)
    se_df['total eggs mixed']= se_df['total eggs mixed'].replace(['26-30'], 6)
    se_df['total eggs mixed']= se_df['total eggs mixed'].replace(['31-35'], 7)
    se_df['total eggs mixed']= se_df['total eggs mixed'].replace(['36-40'], 8)
    se_df['total eggs mixed']= se_df['total eggs mixed'].replace(['>40'], 9)

    se_df['fresh eggs collected']= se_df['fresh eggs collected'].fillna(0)
    se_df['fresh eggs collected']= se_df['fresh eggs collected'].replace(['1-5'], 1)
    se_df['fresh eggs collected']= se_df['fresh eggs collected'].replace(['6-10'], 2)
    se_df['fresh eggs collected']= se_df['fresh eggs collected'].replace(['11-15'], 3)
    se_df['fresh eggs collected']= se_df['fresh eggs collected'].replace(['16-20'], 4)
    se_df['fresh eggs collected']= se_df['fresh eggs collected'].replace(['21-25'], 5)
    se_df['fresh eggs collected']= se_df['fresh eggs collected'].replace(['26-30'], 6)
    se_df['fresh eggs collected']= se_df['fresh eggs collected'].replace(['31-35'], 7)
    se_df['fresh eggs collected']= se_df['fresh eggs collected'].replace(['36-40'], 8)
    se_df['fresh eggs collected']= se_df['fresh eggs collected'].replace(['>40'], 9)
    
    return se_df
    
    

In [5]:
def clean_old_data(tf_df, fs_df):
    old_df= pd.concat([tf_df, fs_df])
    old_df.columns = old_df.columns.str.lower()
    old_df=old_ted_cleaning(old_df)
    old_df= old_df[["patient age at treatment", 'total number of previous ivf cycles','total number of ivf pregnancies',
        'total number of live births - conceived through ivf', "cause  of infertility - tubal disease",
        "cause of infertility - ovulatory disorder", "cause of infertility - male factor",
        "cause of infertility - patient unexplained", "cause of infertility - endometriosis",
        "stimulation used", "egg source", "sperm from", "fresh cycle", "frozen cycle", "eggs thawed",
        "fresh eggs collected", "eggs mixed with partner sperm", "embryos transfered", "number of live births"]]
    old_df.columns = old_df.columns.str.replace(' ', '_')
    old_df['total_number_of_previous_ivf_cycles']= old_df['total_number_of_previous_ivf_cycles'].replace(['>=5'], '5')
    old_df['total_number_of_ivf_pregnancies']= old_df['total_number_of_ivf_pregnancies'].replace(['>=5'], '5')
    old_df['fresh_eggs_collected']= old_df['fresh_eggs_collected'].replace(['> 50'], '50')
    old_df['eggs_mixed_with_partner_sperm']= old_df['eggs_mixed_with_partner_sperm'].replace(['> 50'], '50')
    old_df[["total_number_of_previous_ivf_cycles", "total_number_of_ivf_pregnancies", "fresh_eggs_collected", "eggs_mixed_with_partner_sperm"]] = old_df[["total_number_of_previous_ivf_cycles", "total_number_of_ivf_pregnancies", "fresh_eggs_collected", "eggs_mixed_with_partner_sperm"]].applymap(np.int64)
    old_df[["fresh_cycle", "frozen_cycle", "eggs_thawed", "embryos_transfered"]] = old_df[["fresh_cycle", "frozen_cycle", "eggs_thawed", "embryos_transfered"]].applymap(np.int64)
    old_df.columns = old_df.columns.str.replace('_', ' ')
    old_df.rename(columns={
        'total number of ivf pregnancies': 'total number of previous pregnancies - ivf and di',
        'total number of live births - conceived through ivf': 'total number of previous live births - ivf or di',
        'cause  of infertility - tubal disease' : 'causes of infertility - tubal disease',
        'cause of infertility - ovulatory disorder' : 'causes of infertility - ovulatory disorder',
        'cause of infertility - male factor' : 'causes of infertility - male factor',
        'cause of infertility - patient unexplained' : 'causes of infertility - patient unexplained',
        'cause of infertility - endometriosis': 'causes of infertility - endometriosis',
        'sperm from': 'sperm source',
        'eggs mixed with partner sperm': 'total eggs mixed',
        "embryos transfered": 'embryos transferred'
    }, inplace=True)
    old_df.columns = old_df.columns.str.replace(' ', '_')
    old_df['old_or_new'] = 0
    
    return old_df
    

In [6]:
def clean_new_data(se_df):
    se_df.columns = se_df.columns.str.lower()
    new_df= new_ted_cleaning(se_df)
    new_df.rename(columns={
        'eggs thawed (0/1)': 'eggs thawed'
    }, inplace=True)
    new_df= se_df[["patient age at treatment", 'total number of previous ivf cycles','total number of previous pregnancies - ivf and di',
        'total number of previous live births - ivf or di', "causes of infertility - tubal disease",
        "causes of infertility - ovulatory disorder", "causes of infertility - male factor",
        "causes of infertility - patient unexplained", "causes of infertility - endometriosis",
        "stimulation used", "egg source", "sperm source", "fresh cycle", "frozen cycle", "eggs thawed",
        "fresh eggs collected", "total eggs mixed", "embryos transferred", "number of live births"]]
    new_df.columns = new_df.columns.str.replace(' ', '_')
    new_df['total_number_of_previous_ivf_cycles']= new_df['total_number_of_previous_ivf_cycles'].replace(['>5'], '6')
    new_df['total_number_of_previous_pregnancies_-_ivf_and_di']= new_df['total_number_of_previous_pregnancies_-_ivf_and_di'].replace(['>3'], '4')
    #fill gaps in total_number_of_previous_pregnancies_-_ivf_and_di with 0, assume 0 ig 
    new_df['old_or_new'] = 1
    return new_df



In [7]:
old_df= clean_old_data(tf_df, fs_df)
new_df= clean_new_data(se_df)


C:\Users\rache\AppData\Local\Temp\ipykernel_23880\4212420111.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fs_df['total eggs mixed'] = fs_df['total eggs mixed'].astype(int)
C:\Users\rache\AppData\Local\Temp\ipykernel_23880\4212420111.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fs_df['total eggs mixed'] = np.select(
C:\Users\rache\AppData\Local\Temp\ipykernel_23880\4212420111.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [8]:
df= pd.concat([old_df, new_df])
print(df)

       patient_age_at_treatment total_number_of_previous_ivf_cycles  \
8214                      38-39                                   3   
8215                    18 - 34                                   2   
8216                    18 - 34                                   3   
8217                        999                                   0   
8218                    18 - 34                                   0   
...                         ...                                 ...   
169611                    18-34                                   0   
169612                    18-34                                   1   
169613                    18-34                                   2   
169614                    35-37                                   0   
169615                    18-34                                   0   

        total_number_of_previous_pregnancies_-_ivf_and_di  \
8214                                                  1.0   
8215                     

In [9]:
def cleaning_together(df):
    
    df=df[df["egg_source"] == 'Patient'] 
    df = df.drop('egg_source', axis=1)
    df=df[df["sperm_source"] == 'Partner'] 
    df = df.drop('sperm_source', axis=1)

    #In IVF, injectable medication containing both follicle-stimulating hormone (FSH) and luteinizing hormone (LH) is 
    #injected into females to stimulate more than one egg developing at a time45. It is described as "Stimulation Used" 
    #in the dataset; this study considers only patient records where stimulation is done.

    df=df[df["stimulation_used"] == 1] 
    df = df.drop('stimulation_used', axis=1)

    lab = LabelEncoder()

    df['patient_age_at_treatment'] = df['patient_age_at_treatment'].replace('-', '', regex=True).replace(' ', '', regex=True).astype(int)
    mask = df['patient_age_at_treatment'] != 999
    df = df[mask]
    df['patient_age_at_treatment'] = lab.fit_transform(df['patient_age_at_treatment'])

    df['number_of_live_births'].where(df['number_of_live_births'] <= 1, 1, inplace=True)

    df= df.reset_index()
    df= df.drop(['index'], axis=1)
    df= df.fillna(0)
    #df[["fresh_cycle", "frozen_cycle", "eggs_thawed", "embryos_transfered"]] = balanced_df[["fresh_cycle", "frozen_cycle", "eggs_thawed", "embryos_transfered"]].applymap(np.int64)
    df[["total_number_of_previous_ivf_cycles", "total_number_of_previous_pregnancies_-_ivf_and_di", "fresh_cycle", "frozen_cycle", "embryos_transferred"]] = df[["total_number_of_previous_ivf_cycles", "total_number_of_previous_pregnancies_-_ivf_and_di", "fresh_cycle", "frozen_cycle", "embryos_transferred"]].applymap(np.int64)
    return df

fin_df= cleaning_together(df)


In [10]:
print(fin_df.dtypes)

patient_age_at_treatment                             int64
total_number_of_previous_ivf_cycles                  int64
total_number_of_previous_pregnancies_-_ivf_and_di    int64
total_number_of_previous_live_births_-_ivf_or_di     int64
causes_of_infertility_-_tubal_disease                int64
causes_of_infertility_-_ovulatory_disorder           int64
causes_of_infertility_-_male_factor                  int64
causes_of_infertility_-_patient_unexplained          int64
causes_of_infertility_-_endometriosis                int64
fresh_cycle                                          int64
frozen_cycle                                         int64
eggs_thawed                                          int64
fresh_eggs_collected                                 int64
total_eggs_mixed                                     int64
embryos_transferred                                  int64
number_of_live_births                                int64
old_or_new                                           int

In [11]:
def balance(df):
    live_birth_df= df[df.number_of_live_births == 1]
    num_live_births= live_birth_df.shape[0]
    no_live_birth_df= df[df.number_of_live_births == 0]
    no_live_birth_sample_df= no_live_birth_df.sample(num_live_births)

    balanced_df= pd.concat([live_birth_df, no_live_birth_sample_df])
    return(balanced_df)

balanced_df=balance(fin_df)

Prep for Classifying 

In [12]:
scaled_df = balanced_df.copy() 
  
# apply normalization techniques 
for column in scaled_df.columns: 
    scaled_df[column] = scaled_df[column]  / scaled_df[column].abs().max() 
      
# view normalized data 
display(scaled_df) 


,patient_age_at_treatment,total_number_of_previous_ivf_cycles,total_number_of_previous_pregnancies_-_ivf_and_di,total_number_of_previous_live_births_-_ivf_or_di,causes_of_infertility_-_tubal_disease,causes_of_infertility_-_ovulatory_disorder,causes_of_infertility_-_male_factor,causes_of_infertility_-_patient_unexplained,causes_of_infertility_-_endometriosis,fresh_cycle,frozen_cycle,eggs_thawed,fresh_eggs_collected,total_eggs_mixed,embryos_transferred,number_of_live_births,old_or_new
6,0.0,0.500000,0.2,0.333333,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.222222,0.12,0.666667,1.0,0.0
16,0.0,0.166667,0.2,0.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.333333,0.26,0.666667,1.0,0.0
25,0.2,0.833333,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.555556,0.44,0.666667,1.0,0.0
27,0.0,0.833333,0.2,0.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.111111,0.10,0.666667,1.0,0.0
28,0.2,0.000000,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.222222,0.16,0.666667,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390765,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.666667,0.12,0.000000,0.0,1.0
46182,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.444444,0.00,0.000000,0.0,0.0
25913,0.0,0.000000,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.222222,0.18,0.666667,0.0,0.0
242186,0.8,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.111111,0.06,0.666667,0.0,0.0


In [13]:
y= scaled_df["number_of_live_births"]
X= scaled_df.drop(["number_of_live_births"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.34)

In [14]:
print(X_train)

        patient_age_at_treatment  total_number_of_previous_ivf_cycles  \
345171                       0.0                             0.333333   
390705                       0.2                             0.000000   
349117                       0.2                             0.500000   
266754                       0.6                             0.333333   
61848                        0.6                             0.166667   
...                          ...                                  ...   
184619                       0.0                             0.500000   
255571                       0.8                             0.000000   
71723                        0.0                             0.166667   
316455                       0.4                             0.166667   
107790                       0.0                             0.000000   

        total_number_of_previous_pregnancies_-_ivf_and_di  \
345171                                                0.0   
3